In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class SegmentTree:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, i):
    self[i] = self._def

  def __getitem__(self, i):
    return self.L[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.L[i] = v
    i >>= 1
    while i:
      self.L[i] = self._f(self.L[2 * i], self.L[2 * i + 1])
      i >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

def sol() :
  N, M = map(int, input().split())
  G = [[] for _ in range(N+1)]
  for i, p in enumerate(map(int, input().split())) :
    if p == -1 : continue
    G[p].append(i+1)
  
  IN = [0] * (N+1) #i번 정점에 방문한 시점
  OUT = [0] * (N+1) #i번 정점을 빠져나온 시점
  cnt = 0

  S = [(1, -1)]
  while S:
    x, p = S[-1]

    if IN[x] == 0:
      cnt += 1
      IN[x] = cnt
      for nx in G[x]:
        if nx == p: continue
        S.append((nx, x))
    else:
      S.pop()
      OUT[x] = cnt

  seg = SegmentTree([0] * (N+2), 0, lambda x, y : x + y)
  ans = []
  for _ in range(M) :
    q, *l = map(int, input().split())
    if q == 1 :
      x, w = l
      seg[IN[x]] += w
    elif q == 2 :
      x = l[0]
      ans.append(seg.query(IN[x], OUT[x]))

  sys.stdout.write('\n'.join(map(str, ans)))
  
sol()

### 풀이
- 이번엔 반대로 푸는 문제이다. 특정 직원을 칭찬했을 때 $w$ 만큼 증가시켜놓았다가, \
어떤 상사의 번호가 주워지면 그 상사의 부하 직원에 대한 모든 점수를 증가시키면 된다.